<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# FIA Biomass Modeling

_by Jeremy Freeman (CarbonPlan), September 19, 2020_

This notebook extracts fits predictive biomass growth curves from FIA data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from forests import load, setup, plot, fit

In [3]:
setup.plotting()

In [58]:
df = load.biomass(store='local', states='all')
df = load.biomass_features(store='local', df=df)

In [246]:
df.groupby('type_code').agg(count=('type_code', 'count')).sort_values('count', ascending=False).T

type_code,161.0,503.0,901.0,520.0,185.0,801.0,201.0,406.0,221.0,809.0,...,260.0,280.0,364.0,360.0,129.0,396.0,300.0,320.0,220.0,342.0
count,31778,30892,16426,14146,13713,13405,12967,9577,8055,6139,...,2,2,2,2,1,1,1,1,1,1


In [242]:
inds = df['type_code'] == 361 # 261 is a problem, 941 / 371 is a good example, 341 shows model deviation
x = df[inds]['age']
y = df[inds]['biomass']
f = [df[inds]['tmax'], df[inds]['ppt']]
(
plot.xy(x=x, y=y, color=f[0], xlim=[0,200], ylim=[0,500])
|
plot.xy(x=x, y=y, color=f[1], xlim=[0,200], ylim=[0,500])
).resolve_scale(color='independent')

alt.HConcatChart(...)

In [243]:
model = fit.growth(x=x, y=y, f=f, noise='gamma')
model.result

      fun: 147.75735528757198
 hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.00773355,  0.002359  ,  0.1643258 , -0.00022737,  0.01548131,
        0.0127784 ])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 287
      nit: 36
   status: 0
  success: True
        x: array([5.00053221e+01, 6.93454719e-02, 1.00000000e+00, 3.38437159e-01,
       1.02736558e-02, 7.71485734e+01])

In [244]:
model.r2(x, f, y)

0.05488670616828284

In [245]:
xlim = [0, 100]
ylim= [0, 400]
(
(
plot.xy(x=x, y=y, color=f[0], xlim=xlim, ylim=ylim)
+ plot.line(x=x, y=model.predict(x, f, [90, 50]), color=np.nanpercentile(f[0], 90), xlim=xlim, ylim=ylim)
+ plot.line(x=x, y=model.predict(x, f, [10, 50]), color=np.nanpercentile(f[0], 10), xlim=xlim, ylim=ylim)
)
|
(
plot.xy(x=x, y=y, color=f[1], xlim=xlim, ylim=ylim)
+ plot.line(x=x, y=model.predict(x, f, [50, 10]), color=np.nanpercentile(f[1], 10), xlim=xlim, ylim=ylim)
+ plot.line(x=x, y=model.predict(x, f, [50, 90]), color=np.nanpercentile(f[1], 90), xlim=xlim, ylim=ylim)
)
).resolve_scale(color='independent')

alt.HConcatChart(...)

In [137]:
xlim = [0,150]
ylim= [0,1000]
((plot.xy(x=x, y=y, xlim=xlim, ylim=ylim)
+ plot.line(x=x, y=model.predict(x, f, [50, 50]), xlim=xlim, ylim=ylim))
|
(plot.xy(x=x, y=model.sample(x, f), xlim=xlim, ylim=ylim)
+ plot.line(x=x, y=model.predict(x, f, [50, 50]), xlim=xlim, ylim=ylim)))

alt.HConcatChart(...)

In [201]:
plot.carto(lat=df['lat'], lon=df['lon'], color=df['biomass'], cmap='yellowgreen', clim=(0,200))

alt.Chart(...)

In [193]:
plot.carto(lat=df['lat'], lon=df['lon'], color=df['tmax'], cmap='plasma', clim=(5, 30))

alt.Chart(...)